In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import torch  
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import time
import os

# ✅ Use GPU if available, else fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[CHECKPOINT] 🚀 Using device: {device}")

# ✅ Define Dataset Paths
train_data_path = r"C:/Users/adity/Downloads/Train"
test_data_path = r"C:/Users/adity/Downloads/Test"

# ✅ Verify if paths exist
if not os.path.exists(train_data_path) or not os.path.exists(test_data_path):
    raise FileNotFoundError(f"Dataset folders not found! Check paths:\n{train_data_path}\n{test_data_path}")

# ✅ Transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# ✅ Load Dataset using ImageFolder (Auto-detects class folders)
train_dataset = ImageFolder(root=train_data_path, transform=transform)
test_dataset = ImageFolder(root=test_data_path, transform=transform)

# ✅ Data Loaders (Increased batch size to 128)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

print(f"[CHECKPOINT] ✅ Loaded {len(train_dataset)} training images from {train_data_path}")
print(f"[CHECKPOINT] ✅ Loaded {len(test_dataset)} testing images from {test_data_path}")

# ✅ Define a Simple CNN Model
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 16 * 16, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)  # Dynamically set output classes
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

# ✅ Auto-detect number of classes
num_classes = len(train_dataset.classes)
print(f"[CHECKPOINT] 🔢 Number of classes detected: {num_classes}")

# ✅ Initialize Model
model = CustomCNN(num_classes).to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Training Function
def train_model(model, train_loader, criterion, optimizer, epochs=1):  # Limited to 1 epoch
    model.train()
    print("\n[CHECKPOINT] ✅ Training Started...\n")

    for epoch in range(epochs):
        print(f"\n[CHECKPOINT] 🚀 Epoch {epoch+1}/{epochs} Started...\n")
        start_time = time.time()
        total_loss = 0

        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # ✅ Print every 10 batches for tracking
            if batch_idx % 10 == 0:
                avg_loss = total_loss / (batch_idx + 1)
                print(f"[CHECKPOINT] ✅ Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")

        end_time = time.time()
        print(f"[CHECKPOINT] 🕒 Epoch {epoch+1} Completed in {end_time - start_time:.2f} sec, Final Avg Loss: {total_loss/len(train_loader):.4f}\n")

        # ✅ Save model checkpoint after epoch
        checkpoint_path = f"model_epoch{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"[CHECKPOINT] 💾 Model saved: {checkpoint_path}\n")

    print("[CHECKPOINT] 🎉 Training Complete!\n")

# ✅ Run Training
if __name__ == "__main__":
    print("[CHECKPOINT] ✅ Model Initialized and Ready to Train.")
    train_model(model, train_loader, criterion, optimizer, epochs=1)  # Limited to 1 epoch
    print("[CHECKPOINT] 🎉 Training Finished!")

    # ✅ Save the trained model

[CHECKPOINT] 🚀 Using device: cuda
[CHECKPOINT] ✅ Loaded 1266345 training images from C:/Users/adity/Downloads/Train
[CHECKPOINT] ✅ Loaded 111308 testing images from C:/Users/adity/Downloads/Test
[CHECKPOINT] 🔢 Number of classes detected: 14
[CHECKPOINT] ✅ Model Initialized and Ready to Train.

[CHECKPOINT] ✅ Training Started...


[CHECKPOINT] 🚀 Epoch 1/1 Started...

[CHECKPOINT] ✅ Epoch 1, Batch 0/9894, Loss: 2.6840, Avg Loss: 2.6840
[CHECKPOINT] ✅ Epoch 1, Batch 10/9894, Loss: 2.9850, Avg Loss: 9.4132
[CHECKPOINT] ✅ Epoch 1, Batch 20/9894, Loss: 2.0805, Avg Loss: 6.0440
[CHECKPOINT] ✅ Epoch 1, Batch 30/9894, Loss: 1.4352, Avg Loss: 4.5773
[CHECKPOINT] ✅ Epoch 1, Batch 40/9894, Loss: 1.0652, Avg Loss: 3.7414
[CHECKPOINT] ✅ Epoch 1, Batch 50/9894, Loss: 1.1255, Avg Loss: 3.2126
[CHECKPOINT] ✅ Epoch 1, Batch 60/9894, Loss: 1.0573, Avg Loss: 2.8423
[CHECKPOINT] ✅ Epoch 1, Batch 70/9894, Loss: 0.8318, Avg Loss: 2.5791
[CHECKPOINT] ✅ Epoch 1, Batch 80/9894, Loss: 0.7801, Avg Loss: 2.3779
[C

In [11]:
import torch

def test_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print(f"[CHECKPOINT] 🎯 Test Accuracy: {accuracy:.2f}%")
    print(f"[CHECKPOINT] 🔍 Predicted Labels: {all_preds}")
    print(f"[CHECKPOINT] 🏷️ Actual Labels: {all_labels}")

# ✅ Run Testing
if __name__ == "__main__":
    print("[CHECKPOINT] 🚀 Running Model on Test Data...")
    test_model(model, test_loader, device)
    print("[CHECKPOINT] 🎉 Testing Finished!")


[CHECKPOINT] 🚀 Running Model on Test Data...
[CHECKPOINT] 🎯 Test Accuracy: 57.74%
[CHECKPOINT] 🔍 Predicted Labels: [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,